# Demo: OpenAI Large Language Model - A RAG Sample

In this demo, we show how to use GPT3 model to analyze natural query, use knowledge base to search for more information and answer questions.

In [1]:
import openai
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

CHAT_COMPLETIONS_MODEL = os.getenv('CHAT_COMPLETION_NAME')

### Start with a natural question

In [2]:
input = "Which presendential candidate is likely to win in 2024 election? Explain the reasons."

### Step 1: GPT3: What do I need to to answer this question?

In [3]:
system_prompt=f'''Answer the following questions as best you can. You have access to the following tools:

Web Search: Use the web to find information

You need to follow STRICTLY below format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Research]
Action Input: the input to the action

Begin!
'''
user_prompt = f'''
Question: {input}
Thought:'''

In [4]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])

I should research the current political landscape and potential presidential candidates for the 2024 election.
Action: Research
Action Input: 2024 presidential election candidates and current political analysis


### Step 2: Search web for more details 

In [24]:
# Use Azure BING Search API to get the results
import requests
import json
url = "https://api.bing.microsoft.com/v7.0/search"
headers = {"Ocp-Apim-Subscription-Key": os.getenv('BING_SUBSCRIPTION_KEY')}
params = {"q": "2024 presidential election candidates and current political analysis"}
response = requests.get(url, headers=headers, params=params)

In [25]:
context = ''
for result in response.json()['webPages']['value']:
    context += result['snippet'] + '\n'

### Collected information for further processing

In [26]:
print(context)

Who Are the 2024 Presidential Election Candidates? - The New York Times Advertisement Who’s Running for President in 2024? By Martín González Gómez and Maggie Astor Updated Jan. 21,...
Francis Suarez The Democrats Incumbent Joe Biden Marianne Williamson Dean Phillips Third Party Robert F. Kennedy Jr. Cornel West Jill Stein The Republicans Back to Top Former President of the...
Comment We asked each of the 2024 presidential candidates the same questions about abortion, climate, crime and guns, the economy, education, elections, foreign policy and immigration. If...
08 Nevada Republican presidential caucuses Election news Biden needles Trump in hopes of pushing him off message Colorado voters emphasize January 6 violence as they urge Supreme Court to keep...
By OLIVIA ALAFRIZ 01/27/2024 10:30 AM EST The day after President Joe Biden ramped up his rhetoric on a border deal, former President Donald Trump again tried to undercut the chances of...
Who’s Ahead In Republican Primary Polls? Nat

In [30]:
system_prompt=f'''Answer the following questions as best you can. Use the context provided below:
{context}

Begin!
'''
user_prompt = f'''
Question: {input}\
Answer:'''

### Step 3: GPT3: What do I need to do to answer this question? (Again)

In [31]:
response = openai.ChatCompletion.create(
    engine=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}],
        max_tokens=400,)

print(response['choices'][0]['message']['content'])

It is not possible to accurately predict which presidential candidate is likely to win the 2024 election at this time. The election is still a while away and there are many factors that can influence the outcome, including the candidates' campaigns, political events, and public opinion. Additionally, as of now, the final lineup of candidates is not fully confirmed. It is important to wait for more developments, including debates, primary results, and campaign strategies, before making any predictions about the likely winner.


### GPT3: I now have final answer to the question. 